<a href="https://colab.research.google.com/github/Bilal-Moussaoui/Data_Science_Basics_PY/blob/main/Machine_Learning_Preprocessing_(Complete)_House_Prices_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import helpful libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [6]:
# Pipeline completo para intentrar mejor los resultados.
import numpy as np
# Load the data, and separate the target
train_data = pd.read_csv('/content/sample_data/train.csv')
test_data = pd.read_csv('/content/sample_data/test.csv')

y = train_data.SalePrice

In [7]:
## Revisión de los NaN que hay para ver que hacemos con cada uno en el .csv
def advanced_summary(df: pd.DataFrame, output_name: str = 'df_info.csv', show_entire_data: bool = False) -> pd.DataFrame:
    df_info = []
    for col in df.columns:
        if (df[col].isnull().any() or show_entire_data):
            nonNull = len(df) - np.sum(pd.isna(df[col])) # Saber number of NaN de la columna que evaluamos
            nonNull_percent = nonNull / len(df)
            unique = df[col].nunique()
            colType = str(df[col].dtype)

            df_info.append([col, nonNull, nonNull_percent, unique, colType])

    column_names = ['colName', 'nonNull-values', 'nonNull-percent', 'unique', 'dtype']
    df_info = pd.DataFrame(df_info, columns=column_names)

    df_info.to_csv(output_name, index=False)
    print(output_name)
    print(df_info)

    return df

## Pipeline integrado de Scikit-Learn (Preprocessing)

Estará formado por tres estapas. Debemos pensar que este pipeline se ejecuta para cada uno de los subconjuntos (k-folds) que creemos para la explotación del modelo (con tal de optimizar sus parámetros). Así que debe soportar training and testing por igual.

<br>

### Qué es el pipeline?
El objetivo de tipo **Pipeline** es un objeto unificado que encapsula todo el flujo de trabajo, desde el preprocesamiento de los datos de entrada (X) hasta la predicción final. Esto hace que no hagamos gestión manual ni haya "data leakage".

El pipeline que vamos a definir **full_pipeline** contendrá la receta completa: Ingeniería de Características $\rightarrow$ Imputación/Codificación $\rightarrow$ Regresión.

<br>

#### Transformer de Scikit-Learn
Un Transformer de scikit-learn es cualquier objeto o clase que implementa los métodos .fit() y .transform() (o el método combinado .fit_transform()). Su función principal es preprocesar o modificar los datos para que sean adecuados para un modelo de machine learning.

<br>

#### FunctionTransformer
Método de Scikit-Learn que permite incluir funciones personalizadas de Python , como lo será la lógica del feature_engineer o apply_ordinal_mapping.

<br>

#### ColumnTransformer
Método parra aplicar diferentes pasos del preprocessing que ya ofrece Scikit-Learn (One-Hot-Enconder, SimpleImputer, Codificación Ordinal {.map}...).
Es importante prestar atención al parámetro remainder='passthrough' que le dice al Transformer que evite tratar las columnas que NO se le especifican.

<br>

#### Flujo Lógico de tu full_pipeline

Cuando llamas a full_pipeline.fit(X_train), los datos fluyen de esta manera:

1.  **Datos Originales ($X_{train}$)**
2.  $\downarrow$ **Paso 1: fe_transformer**
    * Aplica feature_engineer() (Crea HouseAge, SinceRemodel).
3.  $\downarrow$ **Paso 2: ordinal_transformer**
    * Aplica apply_ordinal_mapping() (Convierte KitchenQual, etc., a números).
4.  $\downarrow$ **Paso 3: ColumnTransformer**
    * Aplica **Imputación con 0** a columnas de área.
    * Aplica **Imputación con Mediana** a LotFrontage.
    * Aplica **OneHotEncoder** a las columnas nominales restantes.
5.  $\downarrow$ **Paso 3: NumColumnTranformer**
    * Aplica **Estandarización** a las variables numéricas que lo requieran.
    * Aplicar más preprocesado a las numéricas.
6.  $\downarrow$ **Datos Preprocesados Finales**
7.  $\downarrow$ **Paso 4: rf_model (RandomForestRegressor)**
    * Llama a .fit() para entrenar el modelo con los datos limpios y transformados.

In [8]:
# COLUMNAS QUE TRATAREMOS DURANTE EL PIPELINE DIVIDIDAS SEGÚN SU TRATAMIENTO
##---- TIPO DE FEATURE ENGINEERING A APLICAR:
# 1. COLUMNAS A ELIMINAR:
## Sobretodo por tener demasiados UNIQUE (Casi todos) o por tener siempre el mismo valor
## O bien porque con la FE dejan de tener valor y deben eliminarse para evitar MULTICOLINEALIDAD
DROP_COLS = ['Id', 'BsmtUnfSF', 'Utilities', 'Street']
FE_ORIGINAL_COLS_TO_DROP = ['YearBuilt', 'YearRemodAdd']

# 2. COLUMNAS NUEVAS QUE CREAREMOS (ELIMINANDO LAS QUE PARTICIPAN EN LA CREACIÓN)
NEW_COLS = []



##---- TIPO DE ÍMPUTO A APLICAR:
#------------ NUMÉRICAS
# 1. NUMÉRICAS A IMPUTAR CON 0
NUM_ZERO_COLS = [
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF',
    'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea',
    'GarageYrBlt',
]

# 2. NUMÉRICAS A IMPUTAR CON LA MEDIANA
NUM_MEDIAN_COLS = ['LotFrontage']

#------------ CATEGÓRICAS
# 1. CATEGÓRICAS A IMPUTAR CON NA
CAT_NA_COLS = [
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
    'Fence', 'KitchenQual', 'GarageType', 'MiscFeature', 'Alley'
]

# 2. CATEGÓRICAAS A IMPUTAR CON None (Para que sea consistente con la imputación de MasVnrArea)
CAT_NONE_COLS = ['MasVnrType']

# 3. CATEGÓRICAS A IMPUTAR CON la moda
CAT_MODE_COLS = ['MSZoning', 'Electrical', 'SaleType', 'Functional', 'Exterior1st', 'Exterior2nd']



##---- TIPO DE CODIFICACIÓN DE LAS CATEGÓRICAS
# 1. CODIFICACIÓN ORDINAL (MAPPING)
ORDINAL_CODS_COLS = [
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'KitchenQual',
    'ExterQual', 'ExterCond', 'HeatingQC', 'LotShape', 'LandSlope', 'PavedDrive',
    'CentralAir', 'Functional'
]

# 2. CODIFICACIÓN OHE (HAY DOS FLAGS handle_unknown='ignore' & eliminar MULTICOLINEALIDAD)
OHE_CODS_COLS = [
    'MSZoning', 'SaleType', 'Electrical', 'Alley', 'MasVnrType', 'MiscFeature',
    'Exterior1st', 'Exterior2nd', 'Condition1', 'Condition2', 'GarageType',
    'LandContour', 'LotConfig', 'Neighborhood', 'BldgType', 'HouseStyle',
    'RoofStyle', 'RoofMatl', 'Foundation', 'Heating', 'SaleCondition',
    'MoSold', 'YrSold' # -> Añadidas de último momento, són categóricas.
]

# 3. CODIFICACIÓN OHE CON TUPLAS (Avanzado, de momento solo OHE, si se usa, se deben quitar las columnas de OHE_COLS)
## MultiLabelBlizart
MLB_CODS_COLS = [
    'Exterior1st', 'Exterior2nd', 'Condition1', 'Condition2',
]

In [9]:
# Comprobación de que tratemos todas las columnas:
print("NÚMERO CATEGÓRICAS EN EL DATASET:", len(train_data.select_dtypes(include=['object']).columns.tolist()))
print("NÚMERO CATEGÓRICAS QUE TRATAMOS:", len(ORDINAL_CODS_COLS) + len(OHE_CODS_COLS))

print(train_data)

NÚMERO CATEGÓRICAS EN EL DATASET: 43
NÚMERO CATEGÓRICAS QUE TRATAMOS: 43
        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pa

In [61]:
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin

# MAPPING ORDINALES (PARA poder llamar a apply_ordinal_mapping)
quality_map_no_na = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
quality_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
exposure_map = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}
finishing_type_map = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0}
garage_finish_map = {'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0}
lot_shape_map = {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1}
land_slope_map = {'Gtl': 3, 'Mod': 2, 'Sev': 1}
paved_drive_map = {'Y': 3, 'P': 2, 'N': 1}
central_air_map = {'Y': 1, 'N': 0}
pool_quality_map = {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'NA': 0}
fence_quality_map = {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'NA': 0}
functional_map = {'Typ': 7, 'Min1': 6, 'Min2': 5, 'Mod': 4, 'Maj1': 3, 'Maj2': 2, 'Sev': 1, 'Sal': 0}
garage_qual_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}



## FUNCIONES PERSONALIZADAS DEL PIPELINE
# 1. Ingeniería de Características (FE)
def feature_engineer_time(df: pd.DataFrame) -> pd.DataFrame:
    df['HouseAge'] = df['YrSold'] - df['YearBuilt']
    df['SinceRemodel'] = df['YrSold'] - df['YearRemodAdd']
    df['HasGarage'] = df['GarageYrBlt'].apply(lambda x: 0 if pd.isna(x) else 1)
    return df.copy()

# 2. Mapeo Ordinal (Aplicar los mapeos de calidad/orden)
def apply_ordinal_mapping(df: pd.DataFrame) -> pd.DataFrame:
  # Mapeo de columnas con 'NA' que ya fueron imputadas a 'NA'

  # Mapping directos a quality_map
  for col in ['BsmtQual', 'BsmtCond', 'FireplaceQu']:
          df[col] = df[col].map(quality_map)

  for col in ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual']:
      df[col] = df[col].map(quality_map_no_na)


  # Resto de Mapping
  df['LotShape'] = df['LotShape'].map(lot_shape_map)
  df['LandSlope'] = df['LandSlope'].map(land_slope_map)
  df['CentralAir'] = df['CentralAir'].map(central_air_map)
  df['Functional'] = df['Functional'].map(functional_map)
  df['BsmtExposure'] = df['BsmtExposure'].map(exposure_map)
  df['BsmtFinType1'] = df['BsmtFinType1'].map(finishing_type_map)
  df['BsmtFinType2'] = df['BsmtFinType2'].map(finishing_type_map)
  df['GarageFinish'] = df['GarageFinish'].map(garage_finish_map)
  df['PoolQC'] = df['PoolQC'].map(pool_quality_map)
  df['Fence'] = df['Fence'].map(fence_quality_map)
  df['Functional'] = df['Functional'].map(functional_map)

  # ERROR FATAL, antes NO estaban
  df['PavedDrive'] = df['PavedDrive'].map(paved_drive_map)
  df['GarageQual'] = df['GarageQual'].map(garage_qual_map)
  df['GarageCond'] = df['GarageCond'].map(garage_qual_map)

  return df.copy()

# 3. Quitar los outliers: NO ES PARTE DEL PIPELINE, SÓLO SE APLICA A TRAINING SET
def apply_outliers_treatment(df: pd.DataFrame) -> pd.DataFrame:
  ## Esto SOLO se aplica al Training:
  # GrLivArea
  outlier_index = df[(df['GrLivArea'] > 4000) & (df['SalePrice'] < 300000)].index
  print(f"Indices of GrLivArea outliers to be removed: {outlier_index.tolist()}")
  df = df.drop(outlier_index, axis=0)
  print(f"Number of rows after removal: {len(df)}")

  # GarageArea
  outlier_index = df[(df['GarageArea'] > 1200) & (df['SalePrice'] < 300000)].index
  print(f"Indices of GarageArea to be removed: {outlier_index.tolist()}")
  df = df.drop(outlier_index, axis=0)
  print(f"Number of rows after removal: {len(df)}")

  # TotalBsmtSF
  outlier_index = df[(df['TotalBsmtSF'] > 5000) & (df['SalePrice'] < 200000)].index
  print(f"Indices of TotalBsmtSF to be removed: {outlier_index.tolist()}")
  df = df.drop(outlier_index, axis=0)
  print(f"Number of rows after removal: {len(df)}")

  # 1stFlrSF
  outlier_index = df[(df['1stFlrSF'] > 4000) & (df['SalePrice'] < 200000)].index
  print(f"Indices of 1stFlrSF to be removed: {outlier_index.tolist()}")
  df = df.drop(outlier_index, axis=0)
  print(f"Number of rows after removal: {len(df)}")


  ## OUTLLIERS EN LOS QUE CREO EXTRA PARA TESTING (HABRÍA QUE REVISARLO)
  # # BsmtFinSF1
  # # El gráfico scatter: BsmtFinSF1 vs. SalePrice (quinta fila, primera columna) tiene un punto con un área de sótano terminado de Tipo 1 muy grande (más de $2000$ pies cuadrados) y un precio de venta bajo (alrededor de $180,000$).
  # # BsmtFinSF1
  # outlier_index = df[(df['BsmtFinSF1'] > 2000) & (df['SalePrice'] < 200000)].index
  # print(f"Indices of BsmtFinSF1 outliers to be removed: {outlier_index.tolist()}")
  # df = df.drop(outlier_index, axis=0)
  # print(f"Number of rows after removal: {len(df)}")

  # # LotArea
  # # El gráfico scatter: LotArea vs. SalePrice (primera fila, segunda columna) muestra un punto atípico con un área extremadamente grande (más de $100,000$ pies cuadrados) pero un precio de venta bajo (menos de $200,000$).
  # outlier_index = df[(df['LotArea'] > 100000) & (df['SalePrice'] < 200000)].index
  # print(f"Indices of LotArea outliers to be removed: {outlier_index.tolist()}")
  # df = df.drop(outlier_index, axis=0)
  # print(f"Number of rows after removal: {len(df)}")

  return df.copy()


In [62]:
# IMPORTS DE LAS HERRAMIENTAS PARA EL PREPROCESADO
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config
set_config(transform_output='pandas')

# --- 1. Definición del ColumnTransformer de Imputación ---
# TRANSFORMS: Estimadores para IMPUTAR (NUMÉRICAS & CATEGÓRICAS)
zero_imputer = SimpleImputer(strategy='constant', fill_value=0)
median_imputer = SimpleImputer(strategy='median')
na_imputer = SimpleImputer(strategy='constant', fill_value='NA')
none_imputer = SimpleImputer(strategy='constant', fill_value='None')
mode_imputer = SimpleImputer(strategy='most_frequent')

imputation_ct = ColumnTransformer(
    transformers=[
        # ('column_dropper', FunctionTransformer(lambda X: X.drop(columns=DROP_COLS, errors='ignore').copy(), validate=False), DROP_COLS),  -> Es un step del pipeline, no parte del imputer.
        ('zero_impute', zero_imputer, NUM_ZERO_COLS),
        ('median_impute', median_imputer, NUM_MEDIAN_COLS),
        ('na_impute', na_imputer, CAT_NA_COLS),
        ('none_impute', none_imputer, CAT_NONE_COLS),
        ('mode_impute', mode_imputer, CAT_MODE_COLS),

    ],
    remainder='passthrough', # Pasa todas las demás columnas sin imputación (las ya limpias)
    verbose_feature_names_out=False,  # Ha sido un dolor de cabeza, es el responsable de cambiar el nombre a las columnas una vez pasan por los imputers. Hace que en los siguientes steps del preproceso NO se puedan "harcodear" los nombres
    verbose=True
)

# transformed_df = imputation_ct.fit_transform(train_data)
# print(transformed_df.select_dtypes(include=['object']))
# print(len(transformed_df.columns))
# print(len(train_data.columns))

# --- 2. Definición del ColumnTransformer de Codificación ---
# En primer lugar debemos de tratar que tipo de DROP realizará OneHotEncoder:
# Por simplicidad esto aún no lo hacemos, solo hemos indicado drop='first', hay que cambiarlo por drop=drop=drop_values_per_col
# drop_values_per_col = []
# for col in OHE_CODS_COLS:
#   if col in CAT_NONE_COLS:
#     drop_values_per_col.append('None')
#   elif col in CAT_NA_COLS:
#     drop_values_per_col.append('NA')
#   else:
#     drop_values_per_col.append('first') # Aquí puede fallar ya que es posible que el encoder NO entienda este first.
# print(drop_values_per_col, '\n')

encoding_ct = ColumnTransformer(
    transformers=[
        ('ohe_final', OneHotEncoder(handle_unknown='ignore', # Safely handles categories seen in test but not in train (outputs 0s)
                                    sparse_output=False, drop='first'),
                                    OHE_CODS_COLS)
    ],
    remainder='passthrough', # Keeps all other columns (numerical, etc.) untouched
    verbose_feature_names_out=False # Useful for cleaner column names later
).set_output(transform="pandas") # Output a DataFrame with column names

# --- 3. Ensamblaje del Pipeline Final ---
full_pipeline = Pipeline([
    # PASO 1: Ingeniería de Características (Creación de Age, HasGarage)
    ('feature_engineer', FunctionTransformer(feature_engineer_time, validate=False)),

    # PASO 2: Eliminación de Columnas
    ('column_dropper', FunctionTransformer(lambda X: X.drop(columns=DROP_COLS, errors='ignore').copy(), validate=False)),

    # PASO 3: Imputar con NaN (Cero, Mediana, NA, None, Moda)
    ('nan_imputer', imputation_ct),

    # PASO 4: Mapeo Ordinal (Convierte 'NA' a 0, 'Ex' a 5, etc.)
    ('ordinal_mapping', FunctionTransformer(apply_ordinal_mapping, validate=False)),

    # PASO 5: Coficiación (OHE para categoricas sin jerarquía)
    ('encoding', encoding_ct),

    # PASO 6: Modelo de Regresión
    ('rf_model', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=2, random_state=10))
])

print('Estructuración del pipeline realizada correctamente.')

Estructuración del pipeline realizada correctamente.


In [63]:
# Visualización del pipeline
from sklearn import set_config
set_config(display='diagram')
full_pipeline

Pipeline(steps=[('feature_engineer',
                 FunctionTransformer(func=<function feature_engineer_time at 0x791f8e6ae980>)),
                ('column_dropper',
                 FunctionTransformer(func=<function <lambda> at 0x791f8e6ae840>)),
                ('nan_imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('zero_impute',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['MasVnrArea', 'BsmtFinSF1...
                                                   'MasVnrType', 'MiscFeature',
                                                   'Exterior1st', 'Exterior2nd',
                                                   'Condition1', 'Condition2',
                                                   'GarageType', 'LandContour',
                                                   'LotConfig', 'Neighborhood',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'Foundation', 'Heating',
                                                   'SaleCondition', 'MoSold',
                                                   'YrSold'])],
                                   verbose_feature_names_out=False)),
                ('rf_model',
                 RandomForestRegressor(max_depth=20, n_estimators=200,
                                       random_state=10))])

In [64]:
# Validación simple del Pipeline previo a hacer grid-search:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=10)
train_data_without_outliers = apply_outliers_treatment(train_data).copy()

X_train = train_data_without_outliers.drop(columns=['SalePrice']).copy() # Las columnas de drop se manejan en el pipeline
y_train_log = np.log1p(train_data_without_outliers['SalePrice'])

scores = cross_val_score(
    estimator=full_pipeline,
    X=X_train,
    y=y_train_log,
    scoring='neg_mean_squared_error',
    cv=kf,
    n_jobs=-1
  )

# 3. Mostrar el resultado
# El MSE se devuelve como negativo (por la regla de maximización de Scikit-learn),
# así que lo convertimos a positivo y luego le sacamos la raíz cuadrada (RMSE logarítmico).
rmse_scores = np.sqrt(-scores)

print(f"Scores individuales (RMSE Logarítmico): {rmse_scores}")
print(f"Media del RMSE Logarítmico (RMSE-log promedio): {rmse_scores.mean():.4f}")
print(f"Desviación Estándar: {rmse_scores.std():.4f}")

Indices of GrLivArea outliers to be removed: [523, 1298]
Number of rows after removal: 1458
Indices of GarageArea to be removed: [581, 1061, 1190]
Number of rows after removal: 1455
Indices of TotalBsmtSF to be removed: []
Number of rows after removal: 1455
Indices of 1stFlrSF to be removed: []
Number of rows after removal: 1455
Indices of BsmtFinSF1 outliers to be removed: []
Number of rows after removal: 1455
Indices of LotArea outliers to be removed: [53, 249, 313, 335, 384, 451, 457, 706, 1396]
Number of rows after removal: 1446
Scores individuales (RMSE Logarítmico): [0.12623302 0.14391729 0.13120135 0.1593325  0.11957366]
Media del RMSE Logarítmico (RMSE-log promedio): 0.1361
Desviación Estándar: 0.0141


In [39]:
# Validación y Optimización del modelo
from sklearn.model_selection import KFold, GridSearchCV

# 1. Parámetros a probar con el modelo (rf_model)
param_grid = {
    # Número de árboles: más es mejor, pero más lento.
    'rf_model__n_estimators': [500],  # [150, 300, 500],

    # Profundidad máxima: controla el overfitting. None permite que crezca completamente.
    'rf_model__max_depth': [20],  # [10, 20, None]

    # Mínimo de muestras requeridas para dividir un nodo (mayor valor reduce el overfitting).
    'rf_model__min_samples_split': [2]  # [2, 5, 10]
}

In [40]:
# 2. Definir la Validación Cruzada con 5 Folds
kf = KFold(n_splits=5, shuffle=True, random_state=10)

# 3. Configurar el GridSearchCV
grid_search = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,

    # Métrica de scoring: Negación del MSE en escla log. (Escala log hace que el error sea sensible en relación a que tanto afecta al precio,
    # no es lo mismo 10k para un piso de 500k que de 50k!). Hacemos el estimador negado ya que grid_search busca maximizar,
    # por lo tanto, al negar hacemos que nos de el modelo con un menor error MSE!
    scoring='neg_mean_squared_error',

    cv=kf,
    n_jobs=-1,  # Explota al máximo la CPU
    verbose=2
)

In [45]:
# 4. Entrenamiento final, debemos tener los datos de entrenamiento preparados (y logarítmica)
X_train = train_data_without_outliers.drop(columns=['SalePrice']).copy() # Las columnas de drop se manejan en el pipeline
y_train_log = np.log1p(train_data_without_outliers['SalePrice'])

# 5. Entrenar el GridSearch
grid_search.fit(X_train, y_train_log)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[ColumnTransformer] ... (1 of 6) Processing zero_impute, total=   0.0s
[ColumnTransformer] . (2 of 6) Processing median_impute, total=   0.0s
[ColumnTransformer] ..... (3 of 6) Processing na_impute, total=   0.0s
[ColumnTransformer] ... (4 of 6) Processing none_impute, total=   0.0s
[ColumnTransformer] ... (5 of 6) Processing mode_impute, total=   0.0s
[ColumnTransformer] ..... (6 of 6) Processing remainder, total=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=10, shuffle=True),
             estimator=Pipeline(steps=[('feature_engineer',
                                        FunctionTransformer(func=<function feature_engineer_time at 0x791f8e5fa5c0>)),
                                       ('column_dropper',
                                        FunctionTransformer(func=<function <lambda> at 0x791f8e5fade0>)),
                                       ('nan_imputer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('zero_impute',...
                                                                          'BldgType',
                                                                          'HouseStyle',
                                                                          'RoofStyle',
                                                                          'RoofMatl',
                                                                          'Foundation',
                                                                          'Heating',
                                                                          'SaleCondition',
                                                                          'MoSold',
                                                                          'YrSold'])],
                                                          verbose_feature_names_out=False)),
                                       ('rf_model',
                                        RandomForestRegressor(random_state=10))]),
             n_jobs=-1,
             param_grid={'rf_model__max_depth': [20],
                         'rf_model__min_samples_split': [2],
                         'rf_model__n_estimators': [500]},
             scoring='neg_mean_squared_error', verbose=2)

In [46]:
# Ahora que hemos hecho el grid_search toca mostrar los parámetros que minimizan el RMSE:
print("Mejores parámetros:", grid_search.best_params_)

# 1. Mostrar el mejor score (RMSE logarítmico) obtenido en la validación cruzada
# Recuerda que scoring era 'neg_mean_squared_error', así que hay que negarlo y sacar la raíz cuadrada (la escala logarítimica ya la tiene porque el SalePrice lo hemos convertido previamente a ejecutar el gridsearch)
best_rmse_log = np.sqrt(-grid_search.best_score_)
print(f"Mejor RMSE logarítmico en validación cruzada: {best_rmse_log:.4f}")


# 2. Preprocesar los datos de prueba
# Aplicamos el mismo pipeline de preprocesamiento a los test_data
# best_model se trata de un objeto tipo Pipeline que contiene TODOS los pasos del pipeline + el modelo con los mejores estimadores.
# best_model[:-1] se trata de slicing de lista => Queremos que dentro del pipeline se apliquen TODOS los pasos (El preprocessing) exceptuando el último (La ejecución del modelo)
# El objetivo es claro, aplicar al test_data el mismo proprocesado que al train_data para luego predecir!
best_model = grid_search.best_estimator_
X_test_processed = best_model[:-1].transform(test_data.copy())

# 3. Realizar predicciones en los datos de prueba
predictions_log = best_model.named_steps['rf_model'].predict(X_test_processed)

# PASO Alternativo: Podemos pasar directamente el test_data al pipeline, se encargará el mismo de aplicar los pasos de preproceso y luego predecir con el modelo!
# Lo único a tener en cuenta es que NUNCA debemos usar fit_transform, siempre transform porque sino tendremos un Data Leak muy importante.
# predictions_log = best_model.predict(test_data)

print("Predicciones realizadas en escala logarítmica.")
print(predictions_log)

# 4. Transformar predicciones a la escala original
print("\nPredicciones realizadas en escala logarítmica.")
predictions = np.expm1(predictions_log)
print(predictions)

Mejores parámetros: {'rf_model__max_depth': 20, 'rf_model__min_samples_split': 2, 'rf_model__n_estimators': 500}
Mejor RMSE logarítmico en validación cruzada: 0.1377
Predicciones realizadas en escala logarítmica.
[11.73556813 11.93083258 12.11190321 ... 11.95536514 11.63336727
 12.35875579]

Predicciones realizadas en escala logarítmica.
[124936.40431858 151876.95930047 182024.70802093 ... 155648.99501894
 112798.50928904 232990.16053407]


In [47]:
# 5. Crear el archivo de submission
# El archivo de submission debe tener dos columnas: 'Id' y 'SalePrice'
submission_df = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': predictions})

# Guardar el archivo en formato CSV
submission_df.to_csv('submission.csv', index=False)

print("Archivo submission.csv creado exitosamente.")
display(submission_df.head())

Archivo submission.csv creado exitosamente.


,Id,SalePrice
0,1461,124936.404319
1,1462,151876.959300
2,1463,182024.708021
3,1464,179859.330029
4,1465,195673.107032
